In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# データ準備
train = pd.read_csv("./AI_Data/Titanic/train.csv")
test = pd.read_csv("./AI_Data/Titanic/test.csv")

In [11]:
print(train.shape)
train.describe()

(891, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
print(test.shape)
test.describe()

(418, 11)


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [12]:
#欠損データの保管
def kesson_table(df): 
        null_val = df.isnull().sum()
        percent = 100 * df.isnull().sum()/len(df)
        kesson_table = pd.concat([null_val, percent], axis=1)
        kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
        return kesson_table_ren_columns

In [18]:
kesson_table(train)

,欠損数,%
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [16]:
kesson_table(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


<hr><Strong>欠損データ補完<Strong><hr>

In [26]:
# Ageは中央値、Embarkedは、最も多いS、Cabinはいったん無視
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(train["Age"].median())
print("Ageの中央値：" , train["Age"].median())

train["Embarked"] = train["Embarked"].fillna("S")
test["Embarked"] = test["Embarked"].fillna("S")

test["Fare"] = test["Fare"].fillna(train["Fare"].median())
print("Fareの中央値：" , train["Fare"].median())

Ageの中央値： 28.0
Fareの中央値： 14.4542


<hr><Strong>文字データ変換<Strong><hr>

In [29]:
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
train["Embarked"][train["Embarked"] == "S" ] = 0
train["Embarked"][train["Embarked"] == "C" ] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2

C:\Users\SHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\SHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\SHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\SHO\Anaconda3\li

In [62]:
from sklearn import tree
target = train["Survived"].values
features_one = train[["Pclass", "Sex", "Age", "Fare"]].values
features_two = train[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "Embarked"]].values

# 決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)

my_tree_two = tree.DecisionTreeClassifier()
my_tree_two = my_tree_two.fit(features_two, target)

# 「test」の説明変数の値を取得
test_features_1 = test[["Pclass", "Sex", "Age", "Fare"]].values
test_features_2 = test[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "Embarked"]].values
# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction_1 = my_tree_one.predict(test_features_1)
my_prediction_2 = my_tree_two.predict(test_features_2)

In [63]:
my_prediction.shape

(418,)

In [64]:
# PassengerIdを取得
PassengerId = np.array(test["PassengerId"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution_1 = pd.DataFrame(my_prediction_1, PassengerId, columns = ["Survived"])
my_solution_2 = pd.DataFrame(my_prediction_2, PassengerId, columns = ["Survived"])
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("./AI_Data/Titanic/my_tree_one.csv", index_label = ["PassengerId"])
my_solution.to_csv("./AI_Data/Titanic/my_tree_two.csv", index_label = ["PassengerId"])

In [67]:
# testの正解データで正答率を確認
from sklearn.metrics import accuracy_score

acctual = pd.read_csv("./AI_Data/Titanic/gender_submission.csv")
print(accuracy_score(acctual["Survived"], my_solution_1))
print(accuracy_score(acctual["Survived"], my_solution_2))

#acctual.shape
#my_solution.shape

0.8301435406698564
0.7966507177033493


<hr><Strong>ロジスティック回帰<Strong><hr>

In [71]:
from sklearn.linear_model import LogisticRegression
# 訓練
lr = LogisticRegression()
lr.fit(features_one, target)
test_features_1 = test[["Pclass", "Sex", "Age", "Fare"]].values
logisteic_1 = lr.predict(test_features_1)

print(accuracy_score(acctual["Survived"], logisteic_1))

0.9665071770334929


In [72]:
# PassengerIdを取得
PassengerId = np.array(test["PassengerId"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(logisteic_1, PassengerId, columns = ["Survived"])
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("./AI_Data/Titanic/my_logistic_one.csv", index_label = ["PassengerId"])


<hr><Strong>ニューラルネットワーク<Strong><hr>

In [111]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

#目的変数
target = train["Survived"].values
#説明変数
features = train[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "Embarked"]].values

#訓練
NN = MLPClassifier(activation="logistic",solver="sgd",random_state=0,max_iter=100,hidden_layer_sizes=10000)
NN.fit(features, target)
#テストデータ準備
test_features_NN = test[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "Embarked"]].values
#テストデータ予想
NN_1 = NN.predict(test_features_NN)
print(accuracy_score(acctual["Survived"], NN_1))

0.645933014354067
